### Recommendation System

In [8]:
import pandas as pd

#### Data Description:

Unique ID of each anime.
Anime title.
Anime broadcast type, such as TV, OVA, etc.
anime genre.
The number of episodes of each anime.
The average rating for each anime compared to the number of users who gave ratings.

#### Number of community members for each anime.
Objective:
The objective of this assignment is to implement a recommendation system using cosine similarity on an anime dataset. 

#### Dataset:
Use the Anime Dataset which contains information about various anime, including their titles, genres,No.of episodes and user ratings etc.

In [9]:
df = pd.read_csv("anime.csv")
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


#### Data Preprocessing:

Load the dataset into a suitable data structure (e.g., pandas DataFrame).
Handle missing values, if any.
Explore the dataset to understand its structure and attributes.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [11]:
df = df.dropna(subset=['genre', 'rating']) 
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [12]:
df.reset_index(drop=True, inplace=True)

#### Feature Extraction:

Decide on the features that will be used for computing similarity (e.g., genres, user ratings).
Convert categorical features into numerical representations if necessary.
Normalize numerical features if required.

In [13]:
df['genre'] = df['genre'].str.split(', ')

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

In [18]:
mlb = MultiLabelBinarizer()

In [20]:
genre_features = mlb.fit_transform(df['genre'])
genre_features

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
genre_df = pd.DataFrame(genre_features, columns=mlb.classes_)
genre_df

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
final_features = pd.concat([genre_df, df[['rating']].reset_index(drop=True)], axis=1)


In [24]:
final_features.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,rating
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,9.37
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,9.26
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.25
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,9.17
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.16


#### Recommendation System:

Design a function to recommend anime based on cosine similarity.
Given a target anime, recommend a list of similar anime based on cosine similarity scores.
Experiment with different threshold values for similarity scores to adjust the recommendation list size.
Analyze the performance of the recommendation system and identify areas of improvement.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(final_features, final_features)
cosine_sim

array([[1.        , 0.95117833, 0.94026393, ..., 0.95806178, 0.9588302 ,
        0.96196905],
       [0.95117833, 1.        , 0.94603633, ..., 0.94194941, 0.94270491,
        0.94579097],
       [0.94026393, 0.94603633, 1.        , ..., 0.94187252, 0.94262796,
        0.94571377],
       ...,
       [0.95806178, 0.94194941, 0.94187252, ..., 1.        , 0.99999219,
        0.99977998],
       [0.9588302 , 0.94270491, 0.94262796, ..., 0.99999219, 1.        ,
        0.99985508],
       [0.96196905, 0.94579097, 0.94571377, ..., 0.99977998, 0.99985508,
        1.        ]])

In [26]:
def recommend_anime(title, df, cosine_sim, top_n=5):
    idx = df.index[df['name'] == title].tolist()
    if not idx:
        return "Anime not found!"
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    anime_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[anime_indices]

In [28]:
recommendations = recommend_anime("Naruto", df, cosine_sim, top_n=5)
recommendations

615                                    Naruto: Shippuuden
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
Name: name, dtype: object

#### Step 5: Performance & Improvements
Evaluation Ideas:

User validation: Compare recommendations with user watch lists.

Hybrid approach: Combine genre + rating + members.

Dimensionality reduction: Use PCA to reduce feature space and improve performance.

TF-IDF: For textual features like descriptions instead of simple one-hot encoding.

#### Interview Questions

#### What is Collaborative Filtering?

#### 
Collaborative Filtering (CF) recommends items based on user–item interactions (ratings, likes, etc.) rather than content.

Idea: If two users rate similar items similarly, they will like similar new items.

It uses a user–item matrix and finds patterns through similarity or matrix factorization.

#### Difference Between User-Based and Item-Based Collaborative Filtering

In [29]:
comparison = {
    "Aspect": [
        "Core Idea",
        "Process",
        "Example",
        "Advantage",
        "Limitation"
    ],
    "User-Based CF": [
        "Find users similar to the target user",
        "Recommend items liked by similar users",
        "Users who watched Naruto also watched One Piece",
        "Captures user taste patterns",
        "Scalability issues with large number of users"
    ],
    "Item-Based CF": [
        "Find items similar to the target item",
        "Recommend items similar to those the user liked",
        "Since you watched Naruto, you might like Bleach",
        "More stable as item ratings change less often",
        "May ignore new items without enough ratings"
    ]
}

In [30]:
df_comparison = pd.DataFrame(comparison)
df_comparison

,Aspect,User-Based CF,Item-Based CF
0,Core Idea,Find users similar to the target user,Find items similar to the target item
1,Process,Recommend items liked by similar users,Recommend items similar to those the user liked
2,Example,Users who watched Naruto also watched One Piece,"Since you watched Naruto, you might like Bleach"
3,Advantage,Captures user taste patterns,More stable as item ratings change less often
4,Limitation,Scalability issues with large number of users,May ignore new items without enough ratings
